In [2]:
!pip install transformers==3

Looking in indexes: http://ftp.daumkakao.com/pypi/simple


In [1]:
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.utils.rnn as rnn_utils
# from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import time
import datetime


In [2]:
import os

In [3]:
n_devices = torch.cuda.device_count()
torch.cuda.get_device_name()

'Tesla V100-SXM2-32GB'

In [4]:
!which python

/home/ubuntu/anaconda3/envs/pytorch_p36/bin/python


네이버 영화 리뷰데이터 다운로드

In [5]:
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [6]:
train = pd.read_csv('nsmc/ratings_train.txt', sep = '\t')
test = pd.read_csv('nsmc/ratings_test.txt', sep = '\t')

In [7]:
print('train shape {0}, test shape {1}'.format(train.shape, test.shape))

train shape (150000, 3), test shape (50000, 3)


문장별 전처리

In [8]:
document_bert = ['[CLS] ' + str(s) + ' [SEP]' for s in train.document]
document_bert[:5]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]']

tokenize

사전학습된 BERT multilingual model 내 포함되어있는 토크나이저를 활용

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])

['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']


padding

In [11]:
max_len = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = [ np.array(idx + [0]*(max_len - len(idx))) if len(idx) <=max_len else np.array(idx[:max_len]) for idx in input_ids ]


학습 속도를 높이기 위해 실 데이터가 있는 곳과 padding이 있는 곳을 attention에게 알려준다.

In [12]:
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


train -validation set

In [34]:
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['label'].values , random_state = 42, test_size = 0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size = 0.1)

numpy ndarray로 되어있는 input, label, mask들을 torch tensor로 변환해준다.

In [35]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [59]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test set preprocessing

In [43]:
sentences = test['document']
sentences = ['[CLS] ' + str(s) + ' [SEP]' for s in sentences]
labels = test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [44]:
input_ids = [np.array(idx + [0]*(max_len - len(idx))) if len(idx) < max_len else np.array(idx[:max_len]) for idx in input_ids]

In [52]:
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
# 배치 사이즈만큼 데이터를 나누어주는 것
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = batch_size)

모델 학습

In [53]:
device = torch.device('cuda')

분류를 위한 BERT 모델 생성

In [57]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels = 2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

학습 - 스케쥴링

In [13]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)  # 0으로 나누는 것을 방지하기 위한 epsilon 값

epochs = 4

# 총 훈련 스텝
total_steps = len(train_dataloader) * epoch

# lr 조금씩 감소시키는 스케쥴러
# 어느정도 학습이 되었을 떄 lr을 조금씩 감소시켜 조금더 정밀한 학습을 하기 위함.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0,
                                           num_training_steps = total_steps)


NameError: name 'model' is not defined

학습

In [68]:
# accuracy
# 얼마나 잘 맞추었는가?
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    # hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [77]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
# 기존에 저장되어있는 기울기를 초기화한다
model.zero_grad()

for eps in range(epochs):
    
    print("")
    print('======= Epoch {} / {} ========'.format(eps + 1, epochs))
    print('Training...')
    
    # 시작 시간 설정
    t0 = time.time()
    
    # loss 초기화
    total_loss = 0
    
    # 훈련모드로 변경
    model.train()
    
    for step, batch in enumerate(trains_dataloader):
        elapsed = format_time(time.time() - t0)
        print(' batch {:>5,} of {:>5,}      Elapsed {:}.'.format(step, len(train_dataloader), elapsed))
        
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # Forward
        outputs = model(b_input_ids, 
                        token_type_ids = None, 
                        attention_mask = b_input_mask,
                        labels = b_labels
                       )
        
        # 로스 구함
        loss = outputs[0]
        
        # 총 로스 계산
        total_loss += loss.item()
        
        # Backward
        loss.backward()
        
        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()
        
        # 스케줄러로 학습률 감소
        scheduler.step()
        
        # 그래디언트 초기화
        model.zero_grad()
        
    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)
    
    print("")
    print(" Average training loss: {0:2f}".format(avg_train_loss))
    print(" Training epoch took: {:}".format(format_time(time.time() - t0)))
    
    print("")
    print("Running Validation...")
    
    
    t0 = time.time()
    
    # 평가모드로 변경
    model.eval()
    
    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            
            outputs =  model(b_input_ids, 
                            token_type_ids=None,
                            attention_mask=b_input_mask)
            
        # 로스 구함
        logits = outputs[0]
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
        
    print("  Accuracy: {0:2f}".format(eval_accuracy / nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
print("")
print("Training complete!")


======= Epoch 1 / 4 ========
Training...
 batch     0 of 4,219      Elapsed 0:00:00.
 batch     1 of 4,219      Elapsed 0:00:00.
 batch     2 of 4,219      Elapsed 0:00:01.
 batch     3 of 4,219      Elapsed 0:00:01.
 batch     4 of 4,219      Elapsed 0:00:01.
 batch     5 of 4,219      Elapsed 0:00:01.
 batch     6 of 4,219      Elapsed 0:00:01.
 batch     7 of 4,219      Elapsed 0:00:02.
 batch     8 of 4,219      Elapsed 0:00:02.
 batch     9 of 4,219      Elapsed 0:00:02.
 batch    10 of 4,219      Elapsed 0:00:02.
 batch    11 of 4,219      Elapsed 0:00:03.
 batch    12 of 4,219      Elapsed 0:00:03.
 batch    13 of 4,219      Elapsed 0:00:03.
 batch    14 of 4,219      Elapsed 0:00:03.
 batch    15 of 4,219      Elapsed 0:00:04.
 batch    16 of 4,219      Elapsed 0:00:04.
 batch    17 of 4,219      Elapsed 0:00:04.
 batch    18 of 4,219      Elapsed 0:00:04.
 batch    19 of 4,219      Elapsed 0:00:05.
 batch    20 of 4,219      Elapsed 0:00:05.
 batch    21 of 4,219      Elapsed

 batch   186 of 4,219      Elapsed 0:00:44.
 batch   187 of 4,219      Elapsed 0:00:44.
 batch   188 of 4,219      Elapsed 0:00:45.
 batch   189 of 4,219      Elapsed 0:00:45.
 batch   190 of 4,219      Elapsed 0:00:45.
 batch   191 of 4,219      Elapsed 0:00:45.
 batch   192 of 4,219      Elapsed 0:00:45.
 batch   193 of 4,219      Elapsed 0:00:46.
 batch   194 of 4,219      Elapsed 0:00:46.
 batch   195 of 4,219      Elapsed 0:00:46.
 batch   196 of 4,219      Elapsed 0:00:46.
 batch   197 of 4,219      Elapsed 0:00:47.
 batch   198 of 4,219      Elapsed 0:00:47.
 batch   199 of 4,219      Elapsed 0:00:47.
 batch   200 of 4,219      Elapsed 0:00:47.
 batch   201 of 4,219      Elapsed 0:00:48.
 batch   202 of 4,219      Elapsed 0:00:48.
 batch   203 of 4,219      Elapsed 0:00:48.
 batch   204 of 4,219      Elapsed 0:00:48.
 batch   205 of 4,219      Elapsed 0:00:49.
 batch   206 of 4,219      Elapsed 0:00:49.
 batch   207 of 4,219      Elapsed 0:00:49.
 batch   208 of 4,219      Elaps

 batch   373 of 4,219      Elapsed 0:01:28.
 batch   374 of 4,219      Elapsed 0:01:29.
 batch   375 of 4,219      Elapsed 0:01:29.
 batch   376 of 4,219      Elapsed 0:01:29.
 batch   377 of 4,219      Elapsed 0:01:29.
 batch   378 of 4,219      Elapsed 0:01:29.
 batch   379 of 4,219      Elapsed 0:01:30.
 batch   380 of 4,219      Elapsed 0:01:30.
 batch   381 of 4,219      Elapsed 0:01:30.
 batch   382 of 4,219      Elapsed 0:01:30.
 batch   383 of 4,219      Elapsed 0:01:31.
 batch   384 of 4,219      Elapsed 0:01:31.
 batch   385 of 4,219      Elapsed 0:01:31.
 batch   386 of 4,219      Elapsed 0:01:31.
 batch   387 of 4,219      Elapsed 0:01:32.
 batch   388 of 4,219      Elapsed 0:01:32.
 batch   389 of 4,219      Elapsed 0:01:32.
 batch   390 of 4,219      Elapsed 0:01:32.
 batch   391 of 4,219      Elapsed 0:01:33.
 batch   392 of 4,219      Elapsed 0:01:33.
 batch   393 of 4,219      Elapsed 0:01:33.
 batch   394 of 4,219      Elapsed 0:01:33.
 batch   395 of 4,219      Elaps

 batch   560 of 4,219      Elapsed 0:02:13.
 batch   561 of 4,219      Elapsed 0:02:13.
 batch   562 of 4,219      Elapsed 0:02:13.
 batch   563 of 4,219      Elapsed 0:02:13.
 batch   564 of 4,219      Elapsed 0:02:13.
 batch   565 of 4,219      Elapsed 0:02:14.
 batch   566 of 4,219      Elapsed 0:02:14.
 batch   567 of 4,219      Elapsed 0:02:14.
 batch   568 of 4,219      Elapsed 0:02:14.
 batch   569 of 4,219      Elapsed 0:02:15.
 batch   570 of 4,219      Elapsed 0:02:15.
 batch   571 of 4,219      Elapsed 0:02:15.
 batch   572 of 4,219      Elapsed 0:02:15.
 batch   573 of 4,219      Elapsed 0:02:16.
 batch   574 of 4,219      Elapsed 0:02:16.
 batch   575 of 4,219      Elapsed 0:02:16.
 batch   576 of 4,219      Elapsed 0:02:16.
 batch   577 of 4,219      Elapsed 0:02:17.
 batch   578 of 4,219      Elapsed 0:02:17.
 batch   579 of 4,219      Elapsed 0:02:17.
 batch   580 of 4,219      Elapsed 0:02:17.
 batch   581 of 4,219      Elapsed 0:02:17.
 batch   582 of 4,219      Elaps

 batch   747 of 4,219      Elapsed 0:02:57.
 batch   748 of 4,219      Elapsed 0:02:57.
 batch   749 of 4,219      Elapsed 0:02:57.
 batch   750 of 4,219      Elapsed 0:02:58.
 batch   751 of 4,219      Elapsed 0:02:58.
 batch   752 of 4,219      Elapsed 0:02:58.
 batch   753 of 4,219      Elapsed 0:02:58.
 batch   754 of 4,219      Elapsed 0:02:59.
 batch   755 of 4,219      Elapsed 0:02:59.
 batch   756 of 4,219      Elapsed 0:02:59.
 batch   757 of 4,219      Elapsed 0:02:59.
 batch   758 of 4,219      Elapsed 0:03:00.
 batch   759 of 4,219      Elapsed 0:03:00.
 batch   760 of 4,219      Elapsed 0:03:00.
 batch   761 of 4,219      Elapsed 0:03:00.
 batch   762 of 4,219      Elapsed 0:03:00.
 batch   763 of 4,219      Elapsed 0:03:01.
 batch   764 of 4,219      Elapsed 0:03:01.
 batch   765 of 4,219      Elapsed 0:03:01.
 batch   766 of 4,219      Elapsed 0:03:01.
 batch   767 of 4,219      Elapsed 0:03:02.
 batch   768 of 4,219      Elapsed 0:03:02.
 batch   769 of 4,219      Elaps

...

 batch 3,552 of 4,219      Elapsed 0:14:00.
 batch 3,553 of 4,219      Elapsed 0:14:00.
 batch 3,554 of 4,219      Elapsed 0:14:01.
 batch 3,555 of 4,219      Elapsed 0:14:01.
 batch 3,556 of 4,219      Elapsed 0:14:01.
 batch 3,557 of 4,219      Elapsed 0:14:01.
 batch 3,558 of 4,219      Elapsed 0:14:01.
 batch 3,559 of 4,219      Elapsed 0:14:02.
 batch 3,560 of 4,219      Elapsed 0:14:02.
 batch 3,561 of 4,219      Elapsed 0:14:02.
 batch 3,562 of 4,219      Elapsed 0:14:02.
 batch 3,563 of 4,219      Elapsed 0:14:03.
 batch 3,564 of 4,219      Elapsed 0:14:03.
 batch 3,565 of 4,219      Elapsed 0:14:03.
 batch 3,566 of 4,219      Elapsed 0:14:03.
 batch 3,567 of 4,219      Elapsed 0:14:04.
 batch 3,568 of 4,219      Elapsed 0:14:04.
 batch 3,569 of 4,219      Elapsed 0:14:04.
 batch 3,570 of 4,219      Elapsed 0:14:04.
 batch 3,571 of 4,219      Elapsed 0:14:05.
 batch 3,572 of 4,219      Elapsed 0:14:05.
 batch 3,573 of 4,219      Elapsed 0:14:05.
 batch 3,574 of 4,219      Elaps

 batch 3,739 of 4,219      Elapsed 0:14:44.
 batch 3,740 of 4,219      Elapsed 0:14:44.
 batch 3,741 of 4,219      Elapsed 0:14:45.
 batch 3,742 of 4,219      Elapsed 0:14:45.
 batch 3,743 of 4,219      Elapsed 0:14:45.
 batch 3,744 of 4,219      Elapsed 0:14:45.
 batch 3,745 of 4,219      Elapsed 0:14:46.
 batch 3,746 of 4,219      Elapsed 0:14:46.
 batch 3,747 of 4,219      Elapsed 0:14:46.
 batch 3,748 of 4,219      Elapsed 0:14:46.
 batch 3,749 of 4,219      Elapsed 0:14:47.
 batch 3,750 of 4,219      Elapsed 0:14:47.
 batch 3,751 of 4,219      Elapsed 0:14:47.
 batch 3,752 of 4,219      Elapsed 0:14:47.
 batch 3,753 of 4,219      Elapsed 0:14:48.
 batch 3,754 of 4,219      Elapsed 0:14:48.
 batch 3,755 of 4,219      Elapsed 0:14:48.
 batch 3,756 of 4,219      Elapsed 0:14:48.
 batch 3,757 of 4,219      Elapsed 0:14:48.
 batch 3,758 of 4,219      Elapsed 0:14:49.
 batch 3,759 of 4,219      Elapsed 0:14:49.
 batch 3,760 of 4,219      Elapsed 0:14:49.
 batch 3,761 of 4,219      Elaps

 batch 3,926 of 4,219      Elapsed 0:15:28.
 batch 3,927 of 4,219      Elapsed 0:15:29.
 batch 3,928 of 4,219      Elapsed 0:15:29.
 batch 3,929 of 4,219      Elapsed 0:15:29.
 batch 3,930 of 4,219      Elapsed 0:15:29.
 batch 3,931 of 4,219      Elapsed 0:15:30.
 batch 3,932 of 4,219      Elapsed 0:15:30.
 batch 3,933 of 4,219      Elapsed 0:15:30.
 batch 3,934 of 4,219      Elapsed 0:15:30.
 batch 3,935 of 4,219      Elapsed 0:15:31.
 batch 3,936 of 4,219      Elapsed 0:15:31.
 batch 3,937 of 4,219      Elapsed 0:15:31.
 batch 3,938 of 4,219      Elapsed 0:15:31.
 batch 3,939 of 4,219      Elapsed 0:15:31.
 batch 3,940 of 4,219      Elapsed 0:15:32.
 batch 3,941 of 4,219      Elapsed 0:15:32.
 batch 3,942 of 4,219      Elapsed 0:15:32.
 batch 3,943 of 4,219      Elapsed 0:15:32.
 batch 3,944 of 4,219      Elapsed 0:15:33.
 batch 3,945 of 4,219      Elapsed 0:15:33.
 batch 3,946 of 4,219      Elapsed 0:15:33.
 batch 3,947 of 4,219      Elapsed 0:15:33.
 batch 3,948 of 4,219      Elaps

 batch 4,113 of 4,219      Elapsed 0:16:13.
 batch 4,114 of 4,219      Elapsed 0:16:13.
 batch 4,115 of 4,219      Elapsed 0:16:13.
 batch 4,116 of 4,219      Elapsed 0:16:13.
 batch 4,117 of 4,219      Elapsed 0:16:13.
 batch 4,118 of 4,219      Elapsed 0:16:14.
 batch 4,119 of 4,219      Elapsed 0:16:14.
 batch 4,120 of 4,219      Elapsed 0:16:14.
 batch 4,121 of 4,219      Elapsed 0:16:14.
 batch 4,122 of 4,219      Elapsed 0:16:15.
 batch 4,123 of 4,219      Elapsed 0:16:15.
 batch 4,124 of 4,219      Elapsed 0:16:15.
 batch 4,125 of 4,219      Elapsed 0:16:15.
 batch 4,126 of 4,219      Elapsed 0:16:16.
 batch 4,127 of 4,219      Elapsed 0:16:16.
 batch 4,128 of 4,219      Elapsed 0:16:16.
 batch 4,129 of 4,219      Elapsed 0:16:16.
 batch 4,130 of 4,219      Elapsed 0:16:17.
 batch 4,131 of 4,219      Elapsed 0:16:17.
 batch 4,132 of 4,219      Elapsed 0:16:17.
 batch 4,133 of 4,219      Elapsed 0:16:17.
 batch 4,134 of 4,219      Elapsed 0:16:18.
 batch 4,135 of 4,219      Elaps

NameError: name 'avwg_train_loss' is not defined

In [70]:
# 시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for step, batch in enumerate(test_dataloader):
    
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0 :
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,} of {:>5, }.    Elapsed {:}.'.format(step, len(test_dataloader), elapsed))
        
        # 배치 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    with model.zero_grad():

        outputs = model(b_input_ids, 
                       token_type_ids = None, 
                       attention_mask=b_input_mask)

    # loss 
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
        

print("")
print('Accuracy: {0:.2f} '.format(eval_accuracy/nb_eval_steps))
print('Test took: {}'.format(format_time(time.time() - t0)))

135008

In [89]:
outputs[0].flatten()

tensor([0.3260], device='cuda:0', grad_fn=<ViewBackward>)

In [94]:
outputs[1].size()

torch.Size([24, 2])

'<.5'